Needs 3 text files with data, dataset.py, dataloader.py, and models

In [1]:
! pip install pytorch_lightning
! pip install torchmetrics

     |████████████████████████████████| 527 kB 7.7 MB/s 
     |████████████████████████████████| 596 kB 51.6 MB/s 
     |████████████████████████████████| 952 kB 46.5 MB/s 
     |████████████████████████████████| 133 kB 46.7 MB/s 
     |████████████████████████████████| 397 kB 2.1 MB/s 
     |████████████████████████████████| 829 kB 50.8 MB/s 
     |████████████████████████████████| 1.1 MB 44.1 MB/s 
     |████████████████████████████████| 144 kB 55.7 MB/s 
     |████████████████████████████████| 271 kB 66.2 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=1cae87303032e21c073dd4b800dd2a55f2370e7590599b6276f0e0a4b96ee2c0
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfull

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from dataset import wiki_dataset
from dataloader import wiki_dataloader
from feedforward import FeedForward 
from rnn import rnn
import torchmetrics
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
  # Load datasets and dataloader - FEEDFORWARD 
  train = wiki_dataset('wiki.train.txt', training=True, token_map='create')
  valid = wiki_dataset('wiki.valid.txt', training=False, token_map=train.token_map)
  test = wiki_dataset('wiki.test.txt', training=False, token_map=train.token_map)
  datasets = [train,valid,test]

  dataloader = wiki_dataloader(datasets=datasets, batch_size=100)


In [4]:
  # Feedforward
  model = FeedForward(context=train.window,embed_dim=100,vocab_size=len(train.unique_tokens))
  tb_logger = pl_loggers.TensorBoardLogger("FeedForward_logs/", name="ff")
  trainer = pl.Trainer(logger=tb_logger, max_epochs=20, gpus=1,callbacks=[EarlyStopping(monitor='val loss')])
  trainer.fit(model, dataloader)
  result = trainer.test(model, dataloader)
  print(result)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: FeedForward_logs/ff

  | Name  | Type             | Params
-------------------------------------------
0 | embed | Embedding        | 2.9 M 
1 | lin1  | Linear           | 50.1 K
2 | bn1   | BatchNorm1d      | 200   
3 | drop1 | Dropout          | 0     
4 | loss  | CrossEntropyLoss | 0     
5 | lin2  | Linear           | 2.9 M 
-------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.878    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test loss': 4.912854194641113}
--------------------------------------------------------------------------------
[{'test loss': 4.912854194641113}]


In [5]:
  # Load datasets and dataloader - RNN 
  train = wiki_dataset('wiki.train.txt', training=True, token_map='create', window=30)
  valid = wiki_dataset('wiki.valid.txt', training=False, token_map=train.token_map, window=30)
  test = wiki_dataset('wiki.test.txt', training=False, token_map=train.token_map, window=30)
  datasets = [train,valid,test]

  dataloader = wiki_dataloader(datasets=datasets, batch_size=20)

In [8]:
# RNN
model = rnn(n_vocab = len(train.unique_tokens), embedding_size=100, hidden_size=100, num_layers=2)
tb_logger = pl_loggers.TensorBoardLogger("RNN_logs/", name="ff")
trainer = pl.Trainer(logger=tb_logger, max_epochs=20, gpus=1, callbacks=[EarlyStopping(monitor='val loss')])
trainer.fit(model, dataloader)
result = trainer.test(model, dataloader)
print(result)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: RNN_logs/ff

  | Name  | Type             | Params
-------------------------------------------
0 | embed | Embedding        | 2.9 M 
1 | rnn   | RNN              | 40.4 K
2 | fc    | Linear           | 2.9 M 
3 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.838    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

None


In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs/

In [ ]:
!ls /content/lightning_logs/staged_ff

In [ ]:
!rm -rf ./lightning_logs/